In [1]:
import pandas as pd
import numpy as np
import requests
import folium
import config
from math import radians, cos, sin, asin, sqrt
import os.path
from functools import partial
import pyproj
import math
import csv
import pickle

# DATA VISUALISATION
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import asPolygon
from shapely.geometry import asLineString
from shapely.ops import transform
from shapely.ops import cascaded_union
from shapely.geometry import Point
import geopandas as gpd

# models
from sklearn.cluster import KMeans, AffinityPropagation, DBSCAN
import scipy

In [2]:
cluster_gdf = pd.read_csv('../raw_data/B_Accident_data/cluster_gdf.pickle')

In [3]:
cluster_gdf['polygon'] = gpd.GeoSeries.from_wkt(cluster_gdf['polygon'])

In [4]:
cluster_gdf = gpd.GeoDataFrame(cluster_gdf, geometry='polygon')

In [5]:
cluster_gdf = cluster_gdf.drop(columns='Unnamed: 0')

In [6]:
cluster_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 453 entries, 0 to 452
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   cluster  453 non-null    int64   
 1   polygon  453 non-null    geometry
 2   count    453 non-null    int64   
dtypes: geometry(1), int64(2)
memory usage: 10.7 KB


In [7]:
cluster_gdf.crs = {'init': 'epsg:4326'}

/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [8]:
#plotting the hotspots in folium
m = folium.Map(location=[52.52000, 13.4050],tiles='openstreetmap', zoom_start=12)
folium.features.GeoJson(cluster_gdf,
                        style_function= lambda x: {'fillColor': 'red', 'color': 'red'},
                        tooltip=folium.GeoJsonTooltip(fields= ['count'],aliases=["Accident count: "],labels=True)).add_to(m)

In [9]:
m

In [39]:
final_df = pd.read_csv('../raw_data/B_Accident_data/greenlane_df.csv')

In [40]:
final_df.head()

,Unnamed: 0,id,title,subtitle,description,status,holder,coordinator,companyPlanning,dateStart,dateEnd,coordinates
0,0,100579,Grellstraße und Storkower Straße,Sichtbarer Radeln in Prenzlauer Berg,Die Grellstraße und die Storkower Straße verbi...,in planning,"Senatsverwaltung für Umwelt, Verkehr und Klima...",GB infraVelo GmbH,PST GmbH,2. Quartal 2022,2. Quartal 2022,"[(52.5454740742, 13.4279048614), (52.545409291..."
1,1,9010009,Annenstraße,Neue Radstreifen und sicherer über die Straße,Mehr Platz für Radfahrende auf der Annenstraße...,completed,"Senatsverwaltung für Umwelt, Verkehr und Klima...",Bezirksamt Mitte,Alliander Stadtlicht GmbH,3. Quartal 2018,3. Quartal 2019,"[(52.5080012995, 13.4141365999), (52.507901198..."
2,2,9080001,Braunschweig­er Straße (Bau­ab­schnitt 1),Asphaltierung für mehr Fahrkomfort,Die Braunschweiger Straße ist eine Nebenstraße...,completed,"Senatsverwaltung für Umwelt, Verkehr und Klima...",Bezirksamt Neukölln,Asphalta Ingenieurgesellschaft für Verkehrsbau...,1. Quartal 2020,3. Quartal 2021,"[(52.4719552993, 13.4512830001), (52.472135099..."
3,3,9080041,Hermannstraße (Bauabschnitt 1),Sicher unterwegs auf Neuköllner Hauptstraße,Die Hermannstraße ist eine viel befahrene Haup...,under construction,"Senatsverwaltung für Umwelt, Verkehr und Klima...",Bezirksamt Neukölln,NaN,4. Quartal 2021,1. Quartal 2022,"[(52.4719349989, 13.4288895994), (52.471881698..."
4,4,9040041,Hohenzollern­damm,Mehr Raum für Radfahrende in Wilmersdorf,Der Hohenzollerndamm ist eine viel befahrene H...,in planning,"Senatsverwaltung für Umwelt, Verkehr und Klima...",Bezirksamt Charlottenburg-Wilmersdorf,Bezirksamt Charlottenburg-Wilmersdorf,3. Quartal 2022,4. Quartal 2022,"[(52.4907948988, 13.3151869005), (52.490736499..."


In [41]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       284 non-null    int64 
 1   id               284 non-null    int64 
 2   title            284 non-null    object
 3   subtitle         284 non-null    object
 4   description      284 non-null    object
 5   status           282 non-null    object
 6   holder           284 non-null    object
 7   coordinator      284 non-null    object
 8   companyPlanning  178 non-null    object
 9   dateStart        272 non-null    object
 10  dateEnd          272 non-null    object
 11  coordinates      284 non-null    object
dtypes: int64(2), object(10)
memory usage: 26.8+ KB


In [42]:
final_df.coordinates

0      [(52.5454740742, 13.4279048614), (52.545409291...
1      [(52.5080012995, 13.4141365999), (52.507901198...
2      [(52.4719552993, 13.4512830001), (52.472135099...
3      [(52.4719349989, 13.4288895994), (52.471881698...
4      [(52.4907948988, 13.3151869005), (52.490736499...
                             ...                        
279    [(52.5377805993, 13.6134120997), (52.537884998...
280    [(52.6110269992, 13.3149481001), (52.611141299...
281                     [(52.5187968988, 13.4536943007)]
282    [(52.5684324457, 13.4988212917), (52.566696899...
283    [(52.4935085989, 13.4988007994), (52.493521299...
Name: coordinates, Length: 284, dtype: object

In [43]:
from shapely.geometry import shape

final_df['coordinates'] = final_df.apply(lambda: row: shape(row['coordinates']), axis=1)

SyntaxError: invalid syntax (2776185057.py, line 3)

In [33]:
for i in range(284):    
    final_df['coordinates'][i] = final_df['coordinates'][i].strip('[').strip(']')

/tmp/ipykernel_4074/2443422091.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['coordinates'][i] = final_df['coordinates'][i].strip('[').strip(']')


In [34]:
final_df.coordinates

0      (52.5454740742, 13.4279048614), (52.5454092918...
1      (52.5080012995, 13.4141365999), (52.5079011988...
2      (52.4719552993, 13.4512830001), (52.4721350991...
3      (52.4719349989, 13.4288895994), (52.4718816988...
4      (52.4907948988, 13.3151869005), (52.4907364993...
                             ...                        
279    (52.5377805993, 13.6134120997), (52.5378849989...
280    (52.6110269992, 13.3149481001), (52.611141299,...
281                       (52.5187968988, 13.4536943007)
282    (52.5684324457, 13.4988212917), (52.566696899,...
283    (52.4935085989, 13.4988007994), (52.493521299,...
Name: coordinates, Length: 284, dtype: object

In [38]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       284 non-null    int64 
 1   id               284 non-null    int64 
 2   title            284 non-null    object
 3   subtitle         284 non-null    object
 4   description      284 non-null    object
 5   status           282 non-null    object
 6   holder           284 non-null    object
 7   coordinator      284 non-null    object
 8   companyPlanning  178 non-null    object
 9   dateStart        272 non-null    object
 10  dateEnd          272 non-null    object
 11  coordinates      284 non-null    object
dtypes: int64(2), object(10)
memory usage: 26.8+ KB


In [36]:
#final_df['coordinates'] = gpd.GeoSeries.from_wkt(final_df['coordinates'])

In [ ]:
cluster_gdf = gpd.GeoDataFrame(cluster_gdf, geometry='polygon')

In [ ]:
cluster_gdf = cluster_gdf.drop(columns='Unnamed: 0')

In [ ]:
completed = final_df.coordinates[final_df.status == 'completed']
under_construction = final_df.coordinates[final_df.status == 'under construction']
in_planning = final_df.coordinates[final_df.status == 'in planning']
intended = final_df.coordinates[final_df.status == 'intended']

In [ ]:
folium.PolyLine(locations=completed, color='green', weight=7, opacity=0.7, tooltip='completed').add_to(m)
folium.PolyLine(locations=under_construction, color='yellow', weight=7, opacity=0.7, tooltip='under construction').add_to(m)
folium.PolyLine(locations=in_planning, color='orange', weight=7, opacity=0.7, tooltip='in planning').add_to(m)
folium.PolyLine(locations=intended, color='red', weight=7, opacity=0.7, tooltip='intended').add_to(m)

In [ ]:
m